In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("latest Orders line items.xlsx")
df

,Order ID,Product Details
0,10588,"Product ID: 118, Product Qty: 1, Product SKU: ..."
1,10589,"Product ID: 117, Product Qty: 1, Product SKU: ..."
2,10590,"Product ID: 114, Product Qty: 6, Product SKU: ..."
3,10591,"Product ID: 114, Product Qty: 2, Product SKU: ..."
4,10593,"Product ID: 122, Product Qty: 5, Product SKU: ..."
5,10594,"Product ID: 113, Product Qty: 1, Product SKU: ..."
6,10595,"Product ID: 118, Product Qty: 1, Product SKU: ..."
7,10596,"Product ID: 113, Product Qty: 1, Product SKU: ..."
8,10597,"Product ID: 126, Product Qty: 4, Product SKU: ..."


In [3]:
df1 = df[["Order ID"]].join(
    df["Product Details"]
    .str.split("Product ID:", expand=True)
    .stack()
    .reset_index(level=1, drop=True)
    .rename("Split Details")
)

print(df1)

     Order ID                                      Split Details
0      199484                                                   
0      199484   4509, Product Qty: 1, Product SKU: 1SUB-ADQ, ...
0      199484   4515, Product Qty: 1, Product SKU: 1RSVP-PRI,...
0      199484   5975, Product Qty: 1, Product SKU: 1PRI-1302,...
1      199843                                                   
...       ...                                                ...
3365   654230   13888, Product Qty: 1, Product SKU: 1HER-2503...
3366   654265                                                   
3366   654265   13886, Product Qty: 1, Product SKU: 1IHS-MAKE...
3367   654363                                                   
3367   654363   13886, Product Qty: 2, Product SKU: 1IHS-MAKE...

[23519 rows x 2 columns]


In [4]:
df1

,Order ID,Split Details
0,199484,
0,199484,"4509, Product Qty: 1, Product SKU: 1SUB-ADQ, ..."
0,199484,"4515, Product Qty: 1, Product SKU: 1RSVP-PRI,..."
0,199484,"5975, Product Qty: 1, Product SKU: 1PRI-1302,..."
1,199843,
...,...,...
3365,654230,"13888, Product Qty: 1, Product SKU: 1HER-2503..."
3366,654265,
3366,654265,"13886, Product Qty: 1, Product SKU: 1IHS-MAKE..."
3367,654363,


In [5]:

def extract_details(details):
    details_list = details.split(", ")
    extracted = {}
    for item in details_list:
        if "Product ID" in item:
            extracted["Product_ID"] = item.split(":")[1].strip()
        elif "Product Qty" in item:
            extracted["Product_Qty"] = item.split(":")[1].strip()
        elif "Product SKU" in item:
            extracted["Product_SKU"] = item.split(":")[1].strip()
        elif "Product Name" in item:
            extracted["Product_Name"] = item.split(":")[1].strip()
        elif "Product Unit Price" in item:
            extracted["Product_Unit_Price"] = item.split(":")[1].strip()
        elif "Product Total Price" in item:
            extracted["Product_Total_Price"] = item.split(":")[1].strip()
        elif "Product Variation Details" in item:
            extracted["Product_Variation_Details"] = item.split(":")[1].strip()
        elif "Product Weight" in item:
            extracted["Product_Weight"] = item.split(":")[1].strip()
        elif "Product Index" in item:
            extracted["Product_Index"] = item.split(":")[1].strip()
    return extracted


extracted_data = df1["Split Details"].apply(extract_details)

extracted_df = pd.DataFrame(extracted_data.tolist())

final_df = pd.concat([df1["Order ID"].reset_index(drop=True), extracted_df.reset_index(drop=True)],axis = 1)
#final_df = df1[["Order ID"]].join(extracted_df)

print(final_df)


      Order ID Product_Qty  Product_SKU  \
0       199484         NaN          NaN   
1       199484           1     1SUB-ADQ   
2       199484           1    1RSVP-PRI   
3       199484           1    1PRI-1302   
4       199843         NaN          NaN   
...        ...         ...          ...   
23514   654230           1    1HER-2503   
23515   654265         NaN          NaN   
23516   654265           1  1IHS-MAKE25   
23517   654363         NaN          NaN   
23518   654363           2  1IHS-MAKE25   

                                Product_Name Product_Weight  \
0                                        NaN            NaN   
1            Art Doll Quarterly Subscription         0.0000   
2                         Prims Subscription         0.0000   
3                          Prims Summer 2013         0.5000   
4                                        NaN            NaN   
...                                      ...            ...   
23514        In Her Studio Summer 2025 – N

In [6]:
final_df = final_df.drop_duplicates().dropna(subset ="Product_Name" )

In [7]:
final_df

,Order ID,Product_Qty,Product_SKU,Product_Name,Product_Weight,Product_Variation_Details,Product_Unit_Price,Product_Total_Price
1,199484,1,1SUB-ADQ,Art Doll Quarterly Subscription,0.0000,Shipping Destination,44.99,44.99|
2,199484,1,1RSVP-PRI,Prims Subscription,0.0000,Shipping Destination,50.99,50.99|
3,199484,1,1PRI-1302,Prims Summer 2013,0.5000,,14.99,14.99
5,199843,1,5CC-BB23,Character Constructions Clear Stamp Set - Bird...,0.1000,,16.95,16.95|
6,199843,1,5CC-FL08,Character Constructions Clear Stamp Set - Fren...,0.2000,,16.95,16.95|
...,...,...,...,...,...,...,...,...
23512,654230,1,1SUB-SOM,Somerset Studio Subscription,0.0000,Subscription Term (Required),57.97,57.97|
23513,654230,2,1IHS-MAKE25,MADE In Her Studio Volume 1 Pre-Order,0.5000,Shipping Destination,15.99,31.98|
23514,654230,1,1HER-2503,In Her Studio Summer 2025 – New,0.5000,,16.99,16.99
23516,654265,1,1IHS-MAKE25,MADE In Her Studio Volume 1 Pre-Order,0.5000,Shipping Destination,15.99,15.99


In [8]:
final_df.to_csv("Refun lineitems 2022.csv")

In [10]:
df1.to_csv("All Data Final Results Part 1.csv")

In [11]:
df2.to_csv("All Data Final Results Part 2.csv")